<a href="https://colab.research.google.com/github/saritmaitra/Natural-Gas_Paul/blob/master/Sarit_20_deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Installation of ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
# do it for only one time


### Install Dash
!pip install dash==1.4.1 
!pip install dash-auth
!pip install dash-html-components==0.13.2
!pip install dash-core-components==0.39.0
!pip install dash-bootstrap-components
!pip install dash-table==3.1.7 
!pip install dash-daq==0.2.1
!pip install pyforest
#!pip install quandl
#!pip install fredapi
!pip install EIA_python
!pip install ffn
!pip install quandl

In [0]:
# Generate .py file to run with flask
%%writefile Sarit_app.py
from pyforest import *
import datetime, pickle, copy
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import eia, ffn, quandl
import pandas_datareader as web

plt.style.use('ggplot')
from statistics import variance 
#from random import randint
import scipy as sp
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve, auc, classification_report, accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix 
from xgboost import XGBClassifier, XGBRegressor, plot_importance

#from fredapi import Fred

#from termcolor import colored
from sklearn.model_selection import cross_val_score, KFold, cross_validate, train_test_split, TimeSeriesSplit
import xgboost as xgb

import dash
import dash_auth
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table
import dash_bootstrap_components as dbc

USERNAME_PASSWORD = {
    'gasanal': 'KSG@analytics2020',
    'admin':'admin'
}
app = dash.Dash(__name__)
suppress_callback_exceptions=True
app.title = 'Natural Gas'

auth = dash_auth.BasicAuth(
    app,
    USERNAME_PASSWORD
)


# Henry Hub Spot Price, Daily (Release Date: 3/11/2020; Next Release Date: 3/18/2020 )
print('\033[4mHenry Hub Natural Gas Spot Price, Daily (Dollars per Million Btu)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    hh_spot = pd.DataFrame(series_search)
    return hh_spot

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.RNGWHHD.D'
      hh_spot = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(hh_spot))
      return hh_spot;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)

hh_spot = main()
hh_spot = hh_spot.rename({'Henry Hub Natural Gas Spot Price, Daily (Dollars per Million Btu)': 'hh_spot'}, axis = 'columns')
hh_spot = hh_spot.reset_index()
hh_spot['index'] = pd.to_datetime(hh_spot['index'].str[:-3], format='%Y %m%d')
hh_spot['Date']= pd.to_datetime(hh_spot['index']) 
hh_spot.set_index('Date', inplace=True) # setting index column
hh_spot = hh_spot.loc['2000-01-01':,['hh_spot']] # setting date range
hh_spot = hh_spot.astype(float)
#print(hh_spot) 
print('\n')

import plotly.express as px
figures = px.line(x=hh_spot.index, y=hh_spot['hh_spot'], labels={'x':'Date', 'y':'Spot Price'})
figures.show()

# US Coal Prices by Region
print('\033[4mUS Coal Prices by Region\033[0m')
coal = quandl.get("EIA/COAL", authtoken="LSQpgUzwJRoF667ZpzyL") 
coal = coal.loc['2000-01-01':,]

# U.S. Natural Gas Rotary Rigs in Operation, Monthly
print('\033[4mNatural Gas Rotary Rigs in Operation, Monthly (Number of Elements)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    drilling = pd.DataFrame(series_search)
    return drilling

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='PET.E_ERTRRG_XR0_NUS_C.M '
      drilling = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(drilling))
      return drilling;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
  
drilling = main()
drilling = drilling.rename({'U.S. Natural Gas Rotary Rigs in Operation, Monthly (Number of Elements)': 'drilling'}, axis = 'columns')
drilling = drilling.reset_index()
drilling['Date']= pd.to_datetime(drilling['index']) 
drilling.set_index('Date', inplace=True) # setting index column
drilling = drilling.loc['2000-01-01':,['drilling']] # setting date range
drilling = drilling.astype(float)
drilling = drilling.resample('D').ffill().bfill()
drilling = drilling/21
#print(drilling)
#print('\n')

# U.S. Natural Gas Exports (Price), Monthly
print('\033[4mPrice of U.S. Natural Gas Exports (Price), Monthly\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    pr_export = pd.DataFrame(series_search)
    return pr_export

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.N9130US3.M'
      pr_export = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(pr_export))
      return pr_export;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)

pr_export = main()
pr_export = pr_export.rename({'Price of U.S. Natural Gas Exports, Monthly (Dollars per Thousand Cubic Feet)': 'pr_export'}, axis = 'columns')
pr_export = pr_export.reset_index()
pr_export['Date']= pd.to_datetime(pr_export['index']) 
pr_export.set_index('Date', inplace=True) # setting index column
pr_export = pr_export.loc['2000-01-01':,['pr_export']] # setting date range
pr_export = pr_export.astype(float)
pr_export = pr_export.resample('D').bfill().ffill()
pr_export = pr_export/21


# Price of U.S. Natural Gas Imports (Price), Monthly
print('\033[4mPrice of U.S. Natural Gas Imports (Price), Monthly (Dollars per Thousand Cubic Feet)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    price_im = pd.DataFrame(series_search)
    return price_im

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.N9100US3.M'
      price_im = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(price_im))
      return price_im;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
price_im = main()
price_im = price_im.rename({'Price of U.S. Natural Gas Imports, Monthly (Dollars per Thousand Cubic Feet)': 'price_im'}, axis = 'columns')
price_im = price_im.reset_index()
price_im['Date']= pd.to_datetime(price_im['index']) 
price_im.set_index('Date', inplace=True) # setting index column
price_im = price_im.loc['2000-01-01':,['price_im']] # setting date range
price_im = price_im.astype(float)
price_im = price_im.resample('D').ffill().bfill()
price_im = price_im/21

# Price of U.S. Natural Gas Imports (Volume), Monthly
print('\033[4mPrice of U.S. Natural Gas Imports (Volume), Monthly (Dollars per Thousand Cubic Feet)\033[0m')
def retrieve_time_series(api, series_ID):
    """
    Return the time series dataframe, based on API and unique Series ID
    """
    #Retrieve Data By Series ID 
    series_search = api.data_by_series(series=series_ID)
    ##Create a pandas dataframe from the retrieved time series
    vl_import = pd.DataFrame(series_search)
    return vl_import

def main():
    """
    Run main script
    """
    try:
      #Create EIA API using your specific API key
      api_key = "ad819ee5a69e69390eadf300fa168fa8"
      api = eia.API(api_key)
      #Declare desired series ID
      series_ID='NG.N9100US2.M'
      vl_import = retrieve_time_series(api, series_ID)
      #Print the returned dataframe df
      print(type(vl_import))
      return vl_import;
    except Exception as e:
      print("error", e)
      return pd.DataFrame(columns=None)
vl_import = main()
vl_import = vl_import.rename({'U.S. Natural Gas Imports, Monthly (Million Cubic Feet)': 'vl_import'}, axis = 'columns')
vl_import = vl_import.reset_index()
vl_import['Date']= pd.to_datetime(vl_import['index']) 
vl_import.set_index('Date', inplace=True) # setting index column
vl_import = vl_import.loc['2000-01-01':,['vl_import']] # setting date range
vl_import = vl_import.astype(float)
vl_import = vl_import.resample('D').ffill().bfill()
vl_import = vl_import/21

# Natural Gas
print('\033[4mNatural Gas Prices\033[0m')
data = quandl.get("CHRIS/CME_NG1", authtoken="LSQpgUzwJRoF667ZpzyL") 
data = data.loc['2000-01-01':,]
data =data [['Open', 'High', 'Low', 'Last', 'Settle', 'Volume']].copy()
data.rename(columns ={'Last': 'Close', 'Settle': 'Adj Close'}, inplace=True)
print('\n')

# SPOT/OPN/CLOSE/ADJ CLOSE/HIGH/LOW/VOLUME

print('\033[4mCurrent spot price\033[0m')
print(round(hh_spot,2).tail(1))
print('\n')
print('\033[4mOpen price\033[0m')
print(round(data['Open'],2).tail(1))
print('\n')
print('\033[4mClose price\033[0m')
print(round(data['Close'],2).tail(1))
print('\n')
print('\033[4mAdj Close price\033[0m')
print(round(data['Adj Close'],2).tail(1))
print('\n')
print('\033[4mHigh price\033[0m')
print(round(data['High'],2).tail(1))
print('\n')
print('\033[4mLow price\033[0m')
print(round(data['Low'],2).tail(1))
print('\n')
print('\033[4mVolume\033[0m')
print(round(data['Volume'],2).tail(1))
print('\n')

import plotly.graph_objects as go
fig = go.Figure(data=[go.Candlestick(x=data.index,
                                     open=data['Open'],
                                     high=data['High'],
                                     low=data['Low'],
                                     close=data['Close'])])
fig.show()
print('\n')

# 4 WEEKS's Open/Close/High/Low PRICE
ng = round(data[['Open', 'Close', 'High', 'Low']],2).copy()
ng = ng.reset_index()
ng.tail()

# STATISTICS 1

perf = data['Adj Close'].copy().calc_stats()
print('\n')
perf.display()
print('\n')

# STATISTICS 2

# Calculate the daily percentage change which is daily return
dt = data["2017":].copy()
daily_ret = dt['Adj Close'].pct_change().fillna(0)
mean_return = daily_ret.mean()
return_stdev = daily_ret.std()
print('Average daily return %:', round((mean_return*100),2))
print('Average Volatility %:', round((return_stdev*100), 2))
print('\n')

print ('Annualized average return %:', round(((mean_return * 252)*100),2))
print('Annualized volatility %:', round(((return_stdev * np.sqrt(252))*100), 2))
print('\n')

print('\033[4mMaximum Drawdown\033[0m')
# Define a trailing 252 trading day window
window = 186

# Calculate the max drawdown in the past window days for each day 
rolling_max = data['Adj Close'].rolling(window, min_periods=1).max()
daily_drawdown = data['Adj Close']/rolling_max - 1.0

# Calculate the minimum (negative) daily drawdown
max_daily_drawdown = daily_drawdown.rolling(window, min_periods=1).min()

fig = plt.figure(figsize=(15,5))
# Plot the results
daily_drawdown.plot()
max_daily_drawdown.plot()
plt.ylabel('Cumulative returns')
plt.title('Top 10 drawdown periods')
# Show the plot
plt.show()
print('\n')


print('\033[4mSharpe of daily return %\033[0m')
sharpe_ratio = daily_ret.mean() / daily_ret.std()
Sharpe = round(sharpe_ratio,2)*100
print('Sharpe Ratio:', Sharpe)
print('\n')

print('\033[4mAverage Annual Sharpe %\033[0m')
Sharpe_an = round(((252**0.5) * sharpe_ratio),2)*100
print('Annualized Sharpe %:', Sharpe_an)

# STATISTICS 3

print('\033[4mProbability of +/-(1%); +/-(3%); +/-%(5) change in price (Data -> 2000- till date)\033[0m')

print ("The probability of price changes between 1%% and -1%% is %1.2f%% " % 
       (100*daily_ret[(daily_ret > -0.01) & (daily_ret < 0.01)].shape[0] / daily_ret.shape[0]))
print ("The probability of price changes between 3%% and -3%% is %1.2f%% " % 
       (100*daily_ret[(daily_ret > -0.03) & (daily_ret < 0.03)].shape[0] / daily_ret.shape[0]))
print ("The probability of price changes between 5%% and -5%% is %1.2f%% " % 
       (100*daily_ret[(daily_ret > -0.05) & (daily_ret < 0.05)].shape[0] / daily_ret.shape[0]))
print ("The probability of price changes more than 5%% is %1.2f%%" % 
       (100*daily_ret[daily_ret > 0.05].shape[0] / daily_ret.shape[0]))
print ("The probability of price changes less than -5%% is %1.2f%%" % 
       (100*daily_ret[daily_ret < -0.05].shape[0] / daily_ret.shape[0]))
print('\n')

print('\033[4mMinimum price [2000- till date]\033[0m')
print(round(data['Adj Close'].min(),2), data['Adj Close'].idxmin());
print('\033[4mMaximum price [2000- till date]\033[0m')
print(round(data['Adj Close'].max(),2), data['Adj Close'].idxmax());
print('\n')

print('\033[4mMinimum daily % return [2017- till date]\033[0m')
print(round(daily_ret.min(),2)*100, daily_ret.idxmin()); 
print('\033[4mMaximum daily % return [2017- till date]\033[0m')
print(round(daily_ret.max()*100, 2), daily_ret.idxmax());
print('\n')


# Merging relevant features
merge1 = data.join(coal, how='left').ffill().bfill()
merge2 = merge1.join(vl_import, how='left').ffill().bfill()
merge3 = merge2.join(price_im, how='left').ffill().bfill()
merge4 = merge3.join(pr_export, how='left').ffill().bfill()
merge5 = merge4.join(drilling, how='left').ffill().bfill()

Data = merge5.copy()
Data.drop(['Powder River Basin 8,800 Btu, 0.8 SO2',
              'Uinta Basin 11,700 Btu, 0.8 SO2', 
           'High', 'Low', 'Close', 'Volume',
           'Adj Close'], axis=1, inplace=True)

Data['sma2'] = Data.Open.rolling(window=2).mean()
Data['sma20'] = Data['Open'].rolling(window=20).mean()
Data["sma2_1"] = Data.sma2.shift(1)

Data["sma2_increment"] = Data.sma2.diff()  

Data["sma2_1_increment"] = Data.sma2_1.diff()  

Data["open_1"] = Data.Open.shift(1)

Data["open_incr"] = Data.Open -Data.Open.shift(1)

# feature engineering
Data['day_of_week'] = Data.index.dayofweek
Data['day_of_month'] = Data.index.day

# The rows with nulls generated by rolling values will be removed.
Data.dropna(inplace=True)

# creating target variable
Data['target'] = (Data.sma2.shift(-1) - Data.sma2)
Data.dropna(inplace=True)
n = Data.drop(['target'], axis=1) # storing in a dataframe for future use

X1 = np.array(Data.copy().drop(['target', 'Open' ], axis=1));
y1 = np.array(Data['target']).copy();

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.20, 
                                                    random_state=42, shuffle=False);
tscv = TimeSeriesSplit(max_train_size=None, n_splits=10)
#TimeSeriesSplit(max_train_size=0.80, n_splits=10)
for train_samples, test_samples in tscv.split(X1_train):
  #print("TRAIN:", train_samples, "TEST:", test_samples)
  X1_train, X1_test = X1[train_samples], X1[test_samples]
  y1_train, y1_test = y1[train_samples], y1[test_samples]

y1_train = pd.DataFrame(y1_train)
y1_train.rename(columns = {0: 'target'}, inplace=True)

y1_test = pd.DataFrame(y1_test)
y1_test.rename(columns = {0: 'target'}, inplace=True)

#The target variables will be transformed for binary classification. 
# A positive change in the value of prices will be classified as 1 and a non-positive change as 0.

def getBinary(val):
    if val>0:
        return 1
    else:
        return 0

# and the transformation is applied on the test data for later use.
# The train data will be transformed while it is being fit.
y1_test_binary = pd.DataFrame(y1_test["target"].apply(getBinary))

regressor = XGBRegressor(objective='reg:squarederror',
                         n_estimators=100,min_child_weight = 4,
                         subsample= 0.7, silent = 1,nthread = 4,
                         colsample_bytree=1,learning_rate=0.05)
Model = regressor.fit(X1_train, y1_train.target.apply(getBinary))
lg_pred = Model.predict(X1_test)
lg_pred_binary = [1 if yp >=0.5 else 0 for yp in lg_pred]
accuracy = (accuracy_score(y1_test_binary,lg_pred_binary))
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print('\033[4mNext day Open price direction\033[0m')
pred = pd.DataFrame(lg_pred_binary[-1:])
pred.rename(columns = {0: 'forecast'}, inplace=True)
d = Data.tail()
d.reset_index(inplace=True)
d = d.append(pd.DataFrame({'Date': pd.date_range(start=d.Date.iloc[-1], 
                                             periods=3, closed='right')}))
d.set_index('Date', inplace=True)
d = d.tail(1)
pred.index = d.index
pred = pred.assign(day_of_week = lambda x: x.index.day_name())
pred['forecast'].replace([0,1],['Open_down','Open_up'],inplace=True)
pred= pred.reset_index()
print(pred)

# For each day where open price > 0, we set the signal 1 means sell  and < 0, we set signal 0, to buy 
print('\033[4mFor each day where open price = 0 = black arrow head  and 1 = green arrow head \033[0m')
buys = Data.loc[Data['target']  < 0]
sells = Data.loc[Data['target'] >0]

# Plot 
fig = plt.figure(figsize=(20, 5))
plt.plot(Data.index, Data['Open'], color = 'b', lw=2., label='Open')

# Plot the buy and sell signals on the same plot
plt.plot(buys.index, Data.loc[buys.index]['Open'], '^', markersize=5, color='r',
         lw=2., label='Buy')
plt.plot(sells.index, Data.loc[sells.index]['Open'], 'v', markersize=5, color='g',
         lw=2., label='Sell')
plt.ylabel('Price (USD/Mbtu)')
plt.xlabel('Date')
plt.title('Buy and sell signals plot')
plt.legend(loc=0)

# Display everything
plt.show()
print('\n')

## Moving averages crossover Plot 
df1 = data['Adj Close'].copy()
sma5 = df1.rolling(window=5).mean()
sma20 = df1.rolling(window=20).mean()

fig = plt.figure(figsize=(20,6))
plt.plot(df1, color='gray', label='Last')
# Plot the buy and sell signals on the same plot
#  When the shorter-term MA crosses above the longer-term MA (Golden cross), it's a buy signal, it indicates that the trend is shifting up. 
#  When the shorter-term MA crosses below the longer-term MA, it's a sell signal (dead/death cross), it indicates that the trend is shifting down. 
plt.plot(sma5.dropna(), color='r', label = 'sma20')
plt.plot(sma20.dropna(), color='g', label = 'sma100')
plt.ylabel('Price')
plt.xlabel('Date')
plt.suptitle('SMA crossover')
plt.title('Buy -> Shorter MA crosses below longer MA (Deadcross); Sell -> Shorter MA crosses above longer MA (Golden cross)')
plt.legend(loc=0)

# Display everything
plt.show()
print('\n')

## 5 days look-ahead Open price

# Merging relevant features
merge1 = data.join(coal, how='left').ffill().bfill();
merge2 = merge1.join(vl_import, how='left').ffill().bfill();
merge3 = merge2.join(price_im, how='left').ffill().bfill();
merge4 = merge3.join(pr_export, how='left').ffill().bfill();
merge5 = merge4.join(drilling, how='left').ffill().bfill();

df2 = merge5.copy()
df2.drop(['Powder River Basin 8,800 Btu, 0.8 SO2',
              'Uinta Basin 11,700 Btu, 0.8 SO2', 
           'High', 'Low', 'Close', 'Volume',
           'Adj Close'], axis=1, inplace=True)


df2['sma3'] = df2.Open.rolling(window=3).mean();
df2['sma9'] = df2['Open'].rolling(window=9).mean();
df2['sma20'] = df2['Open'].rolling(window=20).mean();

df2['ROC'] = ((df2['Open'] - df2['Open'].shift(5)) / (df2['Open'].shift(5)))*100

# feature engineering
df2['day_of_week'] = df2.index.dayofweek
df2['day_of_month'] = df2.index.day

df2['label'] = df2.Open.shift(-10) 
df2.dropna(inplace=True)

from sklearn.linear_model import LinearRegression

X = np.array(df2.copy().drop(['label'], axis=1));
y = np.array(df2['label']).copy();  # assigning Y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                    random_state=42, shuffle=False);
tscv = TimeSeriesSplit(max_train_size=None, n_splits=10)
for train_samples, test_samples in tscv.split(X_train, y_train):
  X_train, X_test = X[train_samples], X[test_samples]
  y_train, y_test = y[train_samples], y[test_samples]

learner = LinearRegression()

learner.fit(X_train,y_train); #training the linear regression model
open_score=learner.score(X_test,y_test);#testing the linear regression model

y_pred = learner.predict(X_test); 
forecast = y_pred[-10:] #set that will contain the forecasted data
forecast = learner.predict(X_test[-10:]); #set that will contain the forecasted data

print(open_score); print(forecast);

from sklearn import metrics
rmse1 = np.sqrt(metrics.mean_squared_error(y_test, y_pred));

op = y_pred.copy()
op = pd.DataFrame(op)
op = op.tail()
D = data.tail()
op.index = D.index
op.rename(columns = {0: 'predicted'}, inplace=True)
op = pd.concat([round(op['predicted'], 3), data['Open'].tail()], axis=1)
op = op.sort_values(by='Date', ascending=False)
op['accuracy'] = round(op.apply(lambda row: row.predicted / 
                                            row.Open *100, axis = 1),2)
op['accuracy'] = pd.Series(["{0:.2f}%".format(val) for val in op['accuracy']], 
                                     index = op.index)
op = op.assign(day_of_week = lambda x: x.index.day_name())
op = op.reset_index()
print(op)

forecast = pd.DataFrame(forecast);
open_forecast = forecast.copy()
open_forecast = open_forecast.head()

# assigning names to columns
open_forecast.rename(columns = {0: 'open_predict'}, inplace=True)

d = data.tail()
d.reset_index(inplace=True)
d = d.append(pd.DataFrame({'Date': pd.date_range(start=d.Date.iloc[-1], periods=6,
                                             freq = 'B', closed='right')}))
d.set_index('Date', inplace=True)
d = d.tail()

open_forecast.index = d.index
open_forecast = open_forecast.assign(day_of_week = lambda x: x.index.day_name())

open_forecast['diff'] = open_forecast['open_predict'].diff()

open_forecast['direction'] = np.where(open_forecast['diff'] > 0, 1, 0)
open_forecast['direction'].replace([0,1],['Open_down','Open_up'],inplace=True)
open_forecast = open_forecast.round(decimals=2)
open_forecast


import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=op['Date'],
    y=op['Open'],
    name="Actual open price"       # this sets its legend entry
))


fig.add_trace(go.Scatter(
    x=op['Date'],
    y=op['predicted'],
    name="Predicted open price"
))

fig.update_layout(
    title="Actual vs Predicted Open Price",
    #xaxis_title="Date",
    yaxis_title="Price ($/Mbtu)",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.update_layout(
    autosize=False,
    width=800,
    height=400,)
fig.update_layout(legend_orientation="h")
fig.show();

## 5 days look-ahead Close price

# Merging relevant features
merge1 = data.join(coal, how='left').ffill().bfill();
merge2 = merge1.join(vl_import, how='left').ffill().bfill();
merge3 = merge2.join(price_im, how='left').ffill().bfill();
merge4 = merge3.join(pr_export, how='left').ffill().bfill();
merge5 = merge4.join(drilling, how='left').ffill().bfill();

df3 = merge5.copy();
df3.drop(['Powder River Basin 8,800 Btu, 0.8 SO2',
              'Uinta Basin 11,700 Btu, 0.8 SO2'], axis=1, inplace=True);

df3['pct_chng'] = df3['Close'].pct_change();
df3['ROC'] = ((df3['Close'] - df3['Close'].shift(5)) / (df3['Close'].shift(5)))*100;
df3['cum_ret'] = (df3['pct_chng'] + 1).cumprod();
df3['sma2'] = df3['Close'].rolling(window=2).mean();
df3['sma5'] = df3['Close'].rolling(window=5).mean();
df3['sma10'] = df3['Close'].rolling(window=10).mean();
df3['sma20'] = df3['Close'].rolling(window=20).mean();

# Ease of Movement 
dm = ((df3['High'] + df3['Low'])/2) - ((df3['High'].shift(1) + df3['Low'].shift(1))/2)
br = (df3['Volume'] / 100000000) / ((df3['High'] - df3['Low']))

rsi_period = 14 
chg = df3['Close'].diff(1)
gain = chg.mask(chg<0,0)
gain = gain
loss = chg.mask(chg>0,0)
loss = loss
avg_gain = gain.ewm(com = rsi_period - 1, min_periods = rsi_period).mean()
avg_loss = loss.ewm(com = rsi_period - 1, min_periods = rsi_period).mean()
rs = abs(avg_gain/avg_loss)

std = df3['Close'].rolling(window=20).std() 
df3['Upper'] = df3['sma20'] + (std * 2)
df3['Lower'] = df3['sma20'] - (std * 2)

# we also add three new columns ‘ho’ ‘lo’ and ‘gain’
df3['h_o'] = df3['High'] - df3['Open'] # distance between Highest and Opening price
df3['l_o'] = df3['Low'] - df3['Open'] # distance between Lowest and Opening price

# feature engineering
df3['day_of_week'] = df3.index.dayofweek
df3['day_of_month'] = df3.index.day

df3['label'] = df3.Close.shift(-10) 
df3.dropna(inplace=True)

df3 = df3.replace([np.inf, -np.inf], np.nan);
df3.dropna(inplace=True);

X1 = np.array(df3.copy().drop(['label', 'High',	'Low','Open',	
                           'Volume',	'Adj Close'], axis=1));
y1 = np.array(df3['label']).copy();  # assigning Y
    
X1train, X1test, y1train, y1test = train_test_split(X1, y1, test_size = 0.20, 
                                                    random_state=42, shuffle=False);
tscv = TimeSeriesSplit(max_train_size=None, n_splits=10);
for train_samples, test_samples in tscv.split(X1train, y1train):
  X1train, X1test = X1[train_samples], X1[test_samples]
  y1train, y1test = y1[train_samples], y1[test_samples]

model = LinearRegression(); #initializing linear regression model
model.fit(X1train, y1train); #training the linear regression model

y1pred =model.predict(X1test);
c_fore= y1pred[-10:]; #set that will contain the forecasted data
close_score=model.score(X1test,y1test);#testing the linear regression model

print(close_score); print(c_fore);

rmse2 = np.sqrt(metrics.mean_squared_error(y1test, y1pred));

c_fore = pd.DataFrame(c_fore);

close_forecast = c_fore.copy();
close_forecast = close_forecast.head();

# assigning names to columns
close_forecast.rename(columns = {0: 'close_predict'}, inplace=True);
close_forecast = round(close_forecast,2);

close_forecast.index = d.index;
close_forecast = close_forecast.assign(day_of_week = lambda x: x.index.day_name());

close_forecast['diff'] = close_forecast['close_predict'].diff()

close_forecast['direction'] = np.where(close_forecast['diff'] > 0, 1, 0)
close_forecast['direction'].replace([0,1],['Close_down','Close_up'],inplace=True)

cp = y1pred.copy()
cp = pd.DataFrame(cp)
cp = cp.tail()
E = data.tail()
cp.index = E.index
cp.rename(columns = {0: 'predicted'}, inplace=True)
cp = pd.concat([round(cp['predicted'], 3), data['Close'].tail()], axis=1)
cp = cp.sort_values(by='Date', ascending=False)
cp['accuracy'] = round(cp.apply(lambda row: row.predicted / 
                                            row.Close *100, axis = 1),2)
cp['accuracy'] = pd.Series(["{0:.2f}%".format(val) for val in cp['accuracy']], 
                                     index = cp.index)
cp = cp.assign(day_of_week = lambda x: x.index.day_name())
cp = cp.reset_index()


fig = go.Figure()

fig.add_trace(go.Scatter(
    x=cp['Date'],
    y=cp['Close'],
    name="Actual close price"       # this sets its legend entry
))


fig.add_trace(go.Scatter(
    x=cp['Date'],
    y=cp['predicted'],
    name="Predicted close price"
))

fig.update_layout(
    title="Actual vs Predicted Open Price",
    #xaxis_title="Date",
    yaxis_title="Price ($/Mbtu)",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    )
)
fig.update_layout(
    autosize=False,
    width=800,
    height=400,)
fig.update_layout(legend_orientation="h")
fig.show();

h_stack = pd.concat([open_forecast['open_predict'], close_forecast['close_predict']], axis=1)
h_stack = h_stack.assign(day_of_week = lambda x: x.index.day_name())
h_stack = h_stack.reset_index();

NG1 = data.copy()
NG1.reset_index(inplace=True)

NG1['week_no'] = NG1['Date'].dt.week
NG1['month_no'] = NG1['Date'].dt.month
NG1['year'] = NG1['Date'].dt.year
NG1['DayofWeek'] = NG1['Date'].dt.dayofweek

monthly = NG1.pivot_table(values= 'Adj Close', columns = ['year'],aggfunc = np.mean, index = ['month_no'])

monthly.loc[0] = monthly.loc[12,:].shift(1)
monthly.fillna(method='ffill', inplace=True) # filling nan values with previous data
monthly = monthly.sort_index()
print(round(monthly,2))

monthly_change = monthly.pct_change()
monthly_change = monthly_change.drop(monthly_change.index[0])
monthly_change['raise'] = monthly_change[(monthly_change.iloc
                                                              [:,:]>0)].count(axis=1)/(2020-2017)
monthly_change['decline'] = monthly_change[(monthly_change.iloc
                                                              [:,:]<0)].count(axis=1)/(2020-2017)   
monthly_change= round(monthly_change[['raise', 'decline']],2)
monthly_change = monthly_change.reset_index()


#####-------Dashboard--Program---------######

data = data.reset_index()
cp = cp.round(decimals=2)
op = op.round(decimals=2)

colors = {
    'background': '#111222',
    'text': '#7FDBFF'
}


app.layout=html.Div(
    [
     html.Div(
            children='Natural Gas Contract',
            style={
                    'textAlign': 'center',
                    'color': colors['text'],
                    'fontSize':45,
                    'background': '#111222'
                  }
      
            ),
     dcc.Tabs(
         [
          dcc.Tab(
              label='Spot Price',
              style={
                  'backgroundColor':'#111222',
                  'color': colors['text'],
                  'fontSize': 25
              },
              selected_style={
                  'backgroundColor':'rgb(50, 50, 50)',
                  'color': colors['text'],
                  'fontSize':25,
                  'fontWeight':'bold'
              },
              children=[
                html.Div(
                  [
                    dcc.Graph(id='spot price',
                      figure={'data':[
                                      {'x':hh_spot.index,'y':hh_spot['hh_spot'],'type':'line','name':'Spot Price'},
                                      ],
                              'layout':{
                                      #'title':'Spot Price ->' + str(hh_spot.hh_spot.tail(1)),
                                      'annotations':[{
                                          'text':'Spot Price: ' + ((str(hh_spot['hh_spot'].tail(1)).split('Name')[0]).split()[2]) + ' Dollars/ MBtu',
                                          'fontSize':20,
                                          'color':colors['text'],
                                          'showarrow':False,
                                          'align':'right',
                                          'x':1,
                                          'y':1,
                                          'xref':'paper',
                                          'yref':'paper'
                                      },
                                      {
                                          'text':'Date: ' + ((str(hh_spot['hh_spot'].tail(1)).split('Name')[0]).split()[1]),
                                          'color':colors['text'],
                                          'showarrow':False,
                                          'align':'right',
                                          'x':1,
                                          'y':0.9,
                                          'xref':'paper',
                                          'yref':'paper'
                                      }],
                                      'plot_bgcolor':colors['background'],
                                      'paper_bgcolor':colors['background'],
                                      'font':{
                                          'color':colors['text']
                                              },
                                        'xaxis':{
                                            'title':'Date',
                                            'rangeslider': {'visible':True},},
                                        'yaxis':{'title':'Price (USD / Mbtu)'}
                                        #'marginBottom':'10px'
                                      }},
                              config={
                                  'displayModeBar': True,
                                  'displaylogo': False,                                       
                                  'modeBarButtonsToRemove': ['zoom2d', 'hoverCompareCartesian', 'hoverClosestCartesian', 'toggleSpikelines']
                                },)
                
                ],
                style={'width': '100%', 'display': 'inline-block'},
                ),

                
              ]),
          
          dcc.Tab(
              label='Natural Gas',
              style={
                  'backgroundColor':'#111222',
                  'color': colors['text'],
                  'fontSize': 25
              },
              selected_style={
                  'backgroundColor':'rgb(50, 50, 50)',
                  'color': colors['text'],
                  'fontSize':25,
                  'fontWeight':'bold'
              },
              children=[
                html.Div(
                  [
                    dcc.Graph(id='NaturalGas',
                      figure={'data':[{
                                'x':data['Date'],
                                'open':data['Open'],
                                'high':data['High'],
                                'low':data['Low'],
                                'close':data['Close'],
                                'type':'candlestick'
                                    }],
                              'layout':{
                                      'title':"Natural Gas Futures (NG=F)",
                                      'plot_bgcolor':colors['background'],
                                      'paper_bgcolor':colors['background'],
                                      'font':{
                                          'color':colors['text']
                                              },
                                        'yaxis':{'title':'Contract #1 (USD/Mbtu)'}
                                        #'marginBottom':'10px'
                                      }},
                              config={
                                  'displayModeBar': True,
                                  'displaylogo': False,                                       
                                  'modeBarButtonsToRemove': ['zoom2d', 'hoverCompareCartesian', 'hoverClosestCartesian', 'toggleSpikelines']
                                },)
                
                ],
                style={'width': '100%', 'display': 'inline-block'},
                ),

                html.Div(
                [
                  html.Div(
                    children='  LATEST VALUES',
                    style={
                        'textAlign': 'center',
                        'color': colors['text'],
                        'fontSize':20,
                        'height':35,
                        'background': '#111222',
                        #'marginLeft':'10px'
                        }
            
                        ),
                 
                   html.Div(
                    [
                         dbc.Card(
                          [html.Div("Date",className="card-name",style={'marginTop':'0%','marginLeft':'25%'}),
                          html.P(
                             (str(data['Date'].tail(1)).split('Name')[0]).split()[1],
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 'marginLeft':'25%'
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','height':'40%'},
                    ),
                                      
                  
                  html.Div(
                    [
                         dbc.Card(
                         [html.Div("Open Price",className="card-name",style={'marginTop':'0%'}),
                          html.P(
                             round(data['Open'].tail(1),2),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','height':'40%'},
                    ),
                 
                  html.Div(
                    [
                         dbc.Card(
                          [html.Div("Close Price",className="card-name",style={'marginTop':'0%'}),

                          html.P(
                             round(data['Close'].tail(1),2),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','height':'40%'},
                    ),
                 
                 html.Div(
                    [
                         dbc.Card(
                          [html.Div("Adj Close Price",className="card-name",style={'marginTop':'0%'}),

                          html.P(
                            round(data['Adj Close'].tail(1),2),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','height':'40%'},
                    ),
                                    
                    
                ]
                ),
                html.Div(
                 [
                 
                 html.Div(
                    [
                         dbc.Card(
                         [html.Div("High Price",className="card-name",style={'marginTop':'0%','marginLeft':'50%','background': '#111222'}),
                          html.P(
                             round(data['High'].tail(1),2),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 'marginLeft':'50%'
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '50%', 'display': 'inline-block','marginBottom':'0.5%','height':'40%'},
                    ),
                  html.Div(
                    [
                         dbc.Card(
                         [html.Div("Low Price",className="card-name",style={'marginTop':'0%'}),
                          html.P(
                             round(data['Low'].tail(1),2),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 #'marginLeft':'440px'
                             }
                          ),
                           
                           ],
                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','marginBottom':'0.5%','height':'40%'},
                    ),
                 html.Div(
                    [
                         dbc.Card(
                         [html.Div("Volume",className="card-name",style={'marginTop':'0%'}),
                          html.P(
                             data['Volume'].tail(1),
                             style={
                                 'textAlign': 'left',
                                 'fontSize':20,
                                 #'marginLeft':'70px'
                             }
                          ),
                           
                           ],

                          style={'background': '#111222',
                                'color':colors['text'],
                                 'height':80,
                                 'textAlign': 'left',
                                 'fontSize':20}
                      )
                    ],
                    style={'width': '25%', 'display': 'inline-block','marginBottom':'0.5%','height':'40%'},
                    ),
                 ]),
                  
                  html.Div(
                     [
                      html.Div(
                        children='Last 5 Observed Prices',
                        style={
                          'textAlign': 'center',
                          'color': colors['text'],
                          'fontSize':20,
                          'background': '#111222'
                          }
                        ),
         
                      html.Div(
                          [
                          dash_table.DataTable(
                            id='last_openclose',
                            columns=[{"name": 'Date', "id": 'Date','type':'datetime'},
                                     {"name": 'Open', "id": 'Open'},
                                     {"name": 'Close', "id": 'Close'},
                                     {"name": 'High', "id": 'High'},
                                     {"name": 'Low', "id": 'Low'},
                            
                                     ],
                            data= ng.tail().to_dict('rows'),
                            style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                          'fontWeight':'bold',
                                          'color': colors['text'],
                                          'textAlign':'center'
                                          },
                            style_cell={
                                'backgroundColor': 'rgb(50, 50, 50)',
                                'color': colors['text'],
                                'textAlign':'center'
                                },
                            )
                          ],

                        )
                      ]
                      ),

                       #cards 0  
                        html.Div(
                        [
                          html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Avg Sharpe daily %",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round(sharpe_ratio,2)*100],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':22}
                              )
                            ],
                            style={'width': '16%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                                              
                          html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Annualized Sharpe %",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round(((252**0.5) * sharpe_ratio),2)*100],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':22}
                              )
                            ],
                            style={'width': '17%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                          html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Avg. Daily returns %",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round((mean_return*100),2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':22}
                              )
                            ],
                            style={'width': '16%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                        html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Avg. Volatility %",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round((return_stdev*100), 2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':22}
                              )
                            ],
                            style={'width': '13%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                        html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Avg ann return %",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [ round(((mean_return * 252)*100),2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':22}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                                              
                          html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Ann. Volatility %",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round(((return_stdev * np.sqrt(252))*100), 2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':22}
                              )
                            ],
                            style={'width': '15%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            )
                        
                            
                            
                        ]
                        ),
                    #cards 1 
                    html.Div(
                    [
                      html.Div(
                        [
                            dbc.Card(
                              [html.Div("Min daily %\n return (2017 - till date)",className="card-name",style={'marginTop':'0.5%',"white-space": "pre"}),
                              html.P(
                                [str(round(daily_ret.min(),2)*100) +' on '+str(daily_ret.idxmin())],
                                style={
                                    'textAlign': 'center',
                                    'fontSize':20
                                }
                              ),
                              
                              ],
                              style={'background': '#111222',
                                    'color':colors['text'],
                                    'height':115,
                                    'textAlign': 'center',
                                    'fontSize':22}
                          )
                        ],
                        style={'width': '24%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                        ),
                                          
                      html.Div(
                        [
                            dbc.Card(
                              [html.Div("Max daily %\n return (2017 - till date)",className="card-name",style={'marginTop':'0.5%',"white-space": "pre"}),
                              html.P(
                                [str(round(daily_ret.max()*100, 2)) +' on '+str(daily_ret.idxmax())],
                                style={
                                    'textAlign': 'center',
                                    'fontSize':20
                                }
                              ),
                              
                              ],
                              style={'background': '#111222',
                                    'color':colors['text'],
                                    'height':115,
                                    'textAlign': 'center',
                                    'fontSize':22}
                          )
                        ],
                        style={'width': '24%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                        ),
                      html.Div(
                        [
                            dbc.Card(
                              [html.Div("Min. Price\n    (2000 - till date)",className="card-name",style={'marginTop':'0.5%',"white-space": "pre"}),
                              html.P(
                                [str(round(data['Adj Close'].min(),2))+' on '+str(data['Adj Close'].idxmin())],
                                style={
                                    'textAlign': 'center',
                                    'fontSize':20
                                }
                              ),
                              
                              ],
                              style={'background': '#111222',
                                    'color':colors['text'],
                                    'height':115,
                                    'textAlign': 'center',
                                    'fontSize':22}
                          )
                        ],
                        style={'width': '24%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                        ),
                      html.Div(
                        [
                            dbc.Card(
                              [html.Div("Max. Price\n    (2000 - till date)",className="card-name",style={'marginTop':'0.5%',"white-space": "pre"}),
                              html.P(
                                [str(round(data['Adj Close'].max(),2))+' on '+ str(data['Adj Close'].idxmax())],
                                style={
                                    'textAlign': 'center',
                                    'fontSize':20
                                }
                              ),
                              
                              
                              ],
                              style={'background': '#111222',
                                    'color':colors['text'],
                                    'height':115,
                                    'textAlign': 'center',
                                    'fontSize':22}
                          )
                        ],
                        style={'width': '24%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                        ),
                        
                        
                    ]
                    ),
                    #cards 2 
                        html.Div(
                        [
                          html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Probability +/-(1)% price change",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round(100*daily_ret[(daily_ret > -0.01) & (daily_ret < 0.01)].shape[0] / daily_ret.shape[0],2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':19}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                                              
                          html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Probability +/-(3)% price change",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round(100*daily_ret[(daily_ret > -0.03) & (daily_ret < 0.03)].shape[0] / daily_ret.shape[0],2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':19}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                          html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Probability +/-(5)% price change",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round(100*daily_ret[(daily_ret > -0.05) & (daily_ret < 0.05)].shape[0] / daily_ret.shape[0],2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':19}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                          html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Probability > 5% price change ",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round(100*daily_ret[daily_ret > 0.05].shape[0] / daily_ret.shape[0],2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':19}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                        html.Div(
                            [
                                dbc.Card(
                                  [html.Div("Probability < 5% price change",className="card-name",style={'marginTop':'0.5%'}),
                                  html.P(
                                    [round(100*daily_ret[daily_ret < -0.05].shape[0] / daily_ret.shape[0],2)],
                                    style={
                                        'textAlign': 'center',
                                        'fontSize':20
                                    }
                                  ),
                                  
                                  ],
                                  style={'background': '#111222',
                                        'color':colors['text'],
                                        'height':105,
                                        'textAlign': 'center',
                                        'fontSize':19}
                              )
                            ],
                            style={'width': '19%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'40%'},
                            ),
                            
                            
                        ]
                        ),
              ]),
             
          
          dcc.Tab(
              label='Buy/Sell Direction',
              style={
                  'backgroundColor':'#111222',
                  'color': colors['text'],
                  'fontSize':25
              },
              selected_style={
                  'backgroundColor':'rgb(50, 50, 50)',
                  'color': colors['text'],
                  'fontSize':25,
                  'fontWeight':'bold'
              },   
              children=[
                    dcc.Graph(id='BuysSells',
                        figure={'data':[
                          {'x':Data.index,'y':Data['Open'],'type':'line','color':'blue','name':'Open Price'},
                          {'x':buys.index,'y':Data.loc[buys.index]['Open'],'mode':'markers','name':'Buys','marker':{'color':'red'}},
                          {'x':sells.index,'y':Data.loc[sells.index]['Open'],'mode':'markers','name':'Sells','marker':{'color':'green'}},
                          
                      ],
                      'layout':{
                        'title':'Buy (Red) and Sell (Green) Signals',
                        'plot_bgcolor':colors['background'],
                        'paper_bgcolor':colors['background'],
                        'font':{
                            'color':colors['text']
                        },
                        'rang':[-0.6,0.6],
                        'marginBottom':'10px',
                        'paddingTop':'10px',
                        'xaxis':{
                            'title':'Date'
                        },
                        'yaxis':{
                            'title':'Price'
                        }
                              }
                        },
                        config={
                            'displayModeBar': True,
                            'displaylogo': False,                                       
                            'modeBarButtonsToRemove': ['zoom2d', 'hoverCompareCartesian', 'hoverClosestCartesian', 'toggleSpikelines']
                          },
                      ),

                      

                      html.Div(
                        [
                            html.Div(
                                [html.Div(
                                  children='Open Price Forecast',
                                  style={
                                    'textAlign': 'center',
                                    'color': colors['text'],
                                    'fontSize':25,
                                    'background': '#111222',
                                    #'display':'flex',
                                    'width':'100%',
                                    'height':'19%',
                                    'paddingTop':'3.5%'
                                    }
                                  ),
                            dash_table.DataTable(
                              id='open_pred_table',
                              columns=(
                              [{"name": 'Date', "id": 'Date','type':'datetime'},
                               {"name": 'Predicted', "id": 'predicted'},
                               {"name": 'Open', "id": 'Open'},
                               {"name": 'Accuracy', "id": 'accuracy'},
                               {"name": 'Day of Week', "id": 'day_of_week'},
                               ]
                      
                              ),
                              data=(op.to_dict('rows')),
                              editable =True,
                              style_table={
                                  #'maxHeight': '50ex',
                                  #'overflowY': 'scroll',
                                  #'display':'flex',
                                  'width': 633,
                                  'minWidth': 250,
                              },
                              style_cell={
                                  'backgroundColor': 'rgb(50, 50, 50)',
                                  'color': colors['text'],
                                  'textAlign':'center'
                              },
                              style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                            'fontWeight':'bold',
                                            'color': colors['text'],
                                            'textAlign':'center'},
                              
                              
                              ),
                              ]),

                            dcc.Graph(id='open_predicition',
                        figure={'data':[
                          {'x':op['Date'],'y':op['Open'],'type':'line','name':'Actual'},
                          {'x':op['Date'],'y':op['predicted'],'type':'line','name':'Predicted'},
                      ],
                      'layout':{
                        'title':'Last 5 Actual vs Predicted Open Price',
                        'plot_bgcolor':colors['background'],
                        'paper_bgcolor':colors['background'],
                        'font':{
                            'color':colors['text']
                        },
                        'rang':[-0.6,0.6],
                        'marginBottom':'10px',
                        'paddingTop':'10px',
                        'xaxis':{
                            'title':'Date'
                        },
                        'yaxis':{
                            'title':'Price (USD / Mbtu)'
                        },
                        'autoSize':False,
                        'width':700,
                        'height':250
                              }
                              
                        },
                        config={
                            'displayModeBar': False,
                            'displaylogo': False,                                       
                            'modeBarButtonsToRemove': ['zoom2d', 'hoverCompareCartesian', 'hoverClosestCartesian', 'toggleSpikelines']
                          },
                      ),

                        ],
                        style={'width':'100%','display':'flex','marginTop':'0.5%'}
                      ),

                      html.Div(
                        [
                            html.Div(
                                [html.Div(
                                  children='Close Price Forecast',
                                  style={
                                    'textAlign': 'center',
                                    'color': colors['text'],
                                    'fontSize':25,
                                    'background': '#111222',
                                    #'display':'flex',
                                    'width':'100%',
                                    'height':'19%',
                                    'paddingTop':'3.5%'
                                    }
                                  ),
                            dash_table.DataTable(
                              id='close_pred_table',
                              columns=(
                              [{"name": 'Date', "id": 'Date','type':'datetime'},
                               {"name": 'Predicted', "id": 'predicted'},
                               {"name": 'Close', "id": 'Close'},
                               {"name": 'Accuracy', "id": 'accuracy'},
                               {"name": 'Day of Week', "id": 'day_of_week'},
                               ]
                      
                              ),
                              data=(cp.to_dict('rows')),
                              editable =True,
                              style_table={
                                  #'maxHeight': '50ex',
                                  #'overflowY': 'scroll',
                                  #'display':'flex',
                                  'width': 633,
                                  'minWidth': 250,
                              },
                              style_cell={
                                  'backgroundColor': 'rgb(50, 50, 50)',
                                  'color': colors['text'],
                                  'textAlign':'center'
                              },
                              style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                            'fontWeight':'bold',
                                            'color': colors['text'],
                                            'textAlign':'center'},
                              
                              
                              ),
                              ]),

                            dcc.Graph(id='close_predicition',
                        figure={'data':[
                          {'x':cp['Date'],'y':cp['Close'],'type':'line','name':'Actual'},
                          {'x':cp['Date'],'y':cp['predicted'],'type':'line','name':'Predicted'},
                      ],
                      'layout':{
                        'title':'Last 5 Actual vs Predicted Close Price',
                        'plot_bgcolor':colors['background'],
                        'paper_bgcolor':colors['background'],
                        'font':{
                            'color':colors['text']
                        },
                        'rang':[-0.6,0.6],
                        'marginBottom':'10px',
                        'paddingTop':'10px',
                        'xaxis':{
                            'title':'Date'
                        },
                        'yaxis':{
                            'title':'Price (USD / Mbtu)'
                        },
                        'autoSize':False,
                        'width':700,
                        'height':250
                              }
                              
                        },
                        config={
                            'displayModeBar': False,
                            'displaylogo': False,                                       
                            'modeBarButtonsToRemove': ['zoom2d', 'hoverCompareCartesian', 'hoverClosestCartesian', 'toggleSpikelines']
                          },
                      ),

                        ],
                        style={'width':'100%','display':'flex','marginTop':'0.5%'}
                      ),

                      
                      
                      html.Div(
                          [
                           html.Div(
                              [
                                html.Div(
                                  children='Next 5 Days Open/Close price',
                                  style={
                                    'textAlign': 'center',
                                    'color': colors['text'],
                                    'fontSize':20,
                                    'background': '#111222'
                                    }
                                  ),
                                
                                html.Div(
                                    [
                                    html.Div(
                                        [
                                          html.P(["Open Confidence: +%.2f%%" % (open_score * 100.0)],
                                          style={
                                              'marginTop':'0%',
                                              'marginBottom':'0%',
                                              'marginLeft': '58%',
                                              'textAlign': 'left',
                                              'color': colors['text'],
                                              'fontSize':15,
                                              'fontWeight':'bold',
                                              'background': '#111222'
                                              })
                                          
                                        ],
                                        style={'width': '70%', 'display': 'inline-block','marginTop':'0%','marginBottom':'0%','background': '#111222'}
                                    ),
                                    html.Div(
                                        [
                                          html.P(["Close Confidence: +%.2f%%" % (close_score*100)],
                                          style={
                                              'marginTop':'0%',
                                              'marginBottom':'0%',
                                              #'marginRight':'40%',
                                              'textAlign': 'left',
                                              'color': colors['text'],
                                              'fontSize':15,
                                              'fontWeight':'bold',
                                              'background': '#111222'
                                              })
                                        ],
                                        style={'width': '30%', 'display': 'inline-block','marginTop':'0%','marginBottom':'0%','background': '#111222'}
                                    )
                                        
                                ]),
                               
                               html.Div(
                                    [
                                    html.Div(
                                        [
                                          html.P(["Open Squared Error: "+str(round(rmse1,2))],
                                          style={
                                              'marginTop':'0%',
                                              'marginBottom':'0%',
                                              'marginLeft': '58%',
                                              'textAlign': 'left',
                                              'color': colors['text'],
                                              'fontSize':15,
                                              #'fontWeight':'bold',
                                              'background': '#111222'
                                              })
                                          
                                        ],
                                        style={'width': '70%', 'display': 'inline-block','marginTop':'0%','marginBottom':'0%','background': '#111222'}
                                    ),
                                    html.Div(
                                        [
                                          html.P(["Close Squared Error: "+str(round(rmse2,2))],
                                          style={
                                              'marginTop':'0%',
                                              'marginBottom':'0%',
                                              #'marginRight':'40%',
                                              'textAlign': 'left',
                                              'color': colors['text'],
                                              'fontSize':15,
                                              #'fontWeight':'bold',
                                              'background': '#111222'
                                              })
                                        ],
                                        style={'width': '30%', 'display': 'inline-block','marginTop':'0%','marginBottom':'0%','background': '#111222'}
                                    )
                                        
                                ]),
                              
                                
                  
                                html.Div(
                                    [
                                    dash_table.DataTable(
                                      id='next_openclose',
                                      columns=[{"name": 'Date', "id": 'Date','type':'datetime'},
                                              {"name":'Open prediction','id':'open_predict'},
                                              {"name":'Close prediction','id':'close_predict'},
                                              {"name": 'Day of Week', 'id': 'day_of_week'}],
                                      data= h_stack.to_dict('rows'),
                                      style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                                    'fontWeight':'bold',
                                                    'color': colors['text'],
                                                    'textAlign':'center'
                                                    },
                                      style_cell={
                                          'backgroundColor': 'rgb(50, 50, 50)',
                                          'color': colors['text'],
                                          'textAlign':'center'
                                          },
                                      )
                                    ],

                                  )
                                ],
                                style={'width': '50%', 'display': 'inline-block','marginTop':'0.5%'},
                                ),

                                html.Div(
                                  [
                                    html.Div(
                                        children='Opening price direction',
                                        style={
                                            'textAlign': 'center',
                                            'color': colors['text'],
                                            'fontSize':22,
                                            'background': '#111222',
                                            'height':30,
                                            }
                                          ),
                                   html.Div(
                                    [
                                    html.Div(
                                        [
                                          html.P(["Accuracy: %.2f%%" % (accuracy * 100.0)],
                                          style={
                                              'marginTop':'0%',
                                              'marginBottom':'0%',
                                              'marginLeft': '0%',
                                              'textAlign': 'left',
                                              'color': colors['text'],
                                              'fontSize':18,
                                              #'fontWeight':'bold',
                                              'background': '#111222',
                                              'height':30
                                              })
                                          
                                        ],
                                        style={'width': '50%', 'display': 'inline-block','marginTop':'0%','marginBottom':'0%','background': '#111222'}
                                    ),
                                    html.Div(
                                        [
                                          html.P(['Tomorrow: Open Price Direction'],
                                          style={
                                              'marginTop':'0%',
                                              'marginBottom':'0%',
                                              'marginLeft':'0%',
                                              'textAlign': 'left',
                                              'color': colors['text'],
                                              'fontSize':18,
                                              #'fontWeight':'bold',
                                              'background': '#111222',
                                              'height':30
                                              })
                                        ],
                                        style={'width': '50%', 'display': 'inline-block','marginTop':'0%','marginBottom':'0%','background': '#111222'}
                                    )
                                        
                                ]),
                                    html.Div(
                                    [
                                        dash_table.DataTable(
                                            id='open_direction',
                                            columns =(
                                                [
                                                  {'name':'Date', 'id':'Date','type':'datetime'},
                                                 #{'name':'Strategy','id':'price_direction'},
                                                 {'name':'Direction','id':'forecast'},
                                                 {'name': 'Day of Week', 'id': 'day_of_week'},
                                                ]
                                            ),
                                            
                                            data=(pred.to_dict('records')),
                                            style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                                          'fontWeight':'bold',
                                                          'color': colors['text'],
                                                          'textAlign':'center'},
                                            style_cell={
                                                'backgroundColor': 'rgb(50, 50, 50)',
                                                'color': colors['text'],
                                                'textAlign':'center'
                                            },
                                            
                                          
                                        )
                                    ],
                                    )
                                  ],
                                  style={'width': '48%', 'display': 'inline-block','marginTop':'0.5%','marginLeft':'0.5%','height':'35%'},
                                  ),
                          ]
                      )
                      ]
                  ),
          
           dcc.Tab(
              label='SMA Crossover',
              style={
                  'backgroundColor':'#111222',
                  'color': colors['text'],
                  'fontSize':25
              },
              selected_style={
                  'backgroundColor':'rgb(50, 50, 50)',
                  'color': colors['text'],
                  'fontSize':25,
                  'fontWeight':'bold'
              },   
              children=[
                        html.Div(
                          [
                            dcc.Graph(id='SMACrossover',
                            figure={'data':
                              [
                                {'x':df1.index,'y':df1,'type':'line','name':'Close'},
                                {'x':df1.index,'y':sma5,'type':'line','name':'SMA5'},
                                {'x':df1.index,'y':sma20,'type':'line','name':'SMA20'},
                              ],
                              'layout':{
                                  'title':'Simple Moving Average Crossover',
                                  'annotations':[{
                                      'text':'Sell -> Shorter MA crosses above longer MA (Golden cross)',
                                      'color':colors['text'],
                                      'showarrow':False,
                                      'align':'right',
                                      'x':1,
                                      'y':1,
                                      'xref':'paper',
                                      'yref':'paper'
                                  },
                                  {
                                      'text':'Buy -> Shorter MA crosses below longer MA (Dead Cross)',
                                      'color':colors['text'],
                                      'showarrow':False,
                                      'align':'right',
                                      'x':1,
                                      'y':0.9,
                                      'xref':'paper',
                                      'yref':'paper'
                                  }],
                                  'plot_bgcolor':colors['background'],
                                  'paper_bgcolor':colors['background'],
                                  'font':{
                                      'color':colors['text']
                                  },
                                  'rang':[-0.6,0.6],
                                  'marginBottom':'10%',
                                  'xaxis':{
                                      'title':'Date'
                                  },
                                  'yaxis':{
                                      'title':'Price'
                                  }
                              }
                              },
                              config={
                                'displayModeBar': True,
                                'displaylogo': False,                                       
                                'modeBarButtonsToRemove': ['zoom2d', 'hoverCompareCartesian', 'hoverClosestCartesian', 'toggleSpikelines']
                              },)                        
                                              
                                              
                            ]),
                        
                       
                        ],
              ),
          
           dcc.Tab(
              label='Drawdown',
              style={
                  'backgroundColor':'#111222',
                  'color': colors['text'],
                  'fontSize':25
              },
              selected_style={
                  'backgroundColor':'rgb(50, 50, 50)',
                  'color': colors['text'],
                  'fontSize':25,
                  'fontWeight':'bold'
              },   
              children=[
                        html.Div(
                          [
                            dcc.Graph(id='DailyDrawdown',
                                      figure={'data':[{
                                            'x':daily_drawdown.index,
                                            'y':daily_drawdown,
                                            'type':'line'
                                        },
                                        
                                        {
                                            'x':max_daily_drawdown.index,
                                            'y':max_daily_drawdown,
                                            'type':'line'
                                        }
                                    ],'layout':{
                                            'title':'Top 10 drawdown periods',
                                            'plot_bgcolor':colors['background'],
                                            'paper_bgcolor':colors['background'],
                                            'font':{
                                                'color':colors['text']
                                            },
                                            'marginBottom':'10px',
                                            'yaxis':{
                                                'title':'Cumulative returns'
                                            }            
                                        }},
                                      config={
                                        'displayModeBar': True,
                                        'displaylogo': False,                                       
                                        'modeBarButtonsToRemove': ['zoom2d', 'hoverCompareCartesian', 'hoverClosestCartesian', 'toggleSpikelines']
                                      },
                                      )                        
                          ],
                          style={'width': '100%', 'display': 'inline-block'},
                        ),

                        
                        html.Div(
                          [
                            html.Div(
                              children='Monthly Gas Price Decline/Raise',
                              style={
                                  'textAlign': 'center',
                                  'color': colors['text'],
                                  'fontSize':20,
                                  'background': '#111222'
                                  }
                              ),
                            html.Div(
                            [
                                dash_table.DataTable(
                                    id='raise_decline',
                                    columns=(
                                    [{"name": 'Month', "id": 'month_no'},
                                    {"name": 'Raise', "id": 'raise'},
                                    {"name": 'Decline', "id": 'decline'},
                                    ]
                                    ),
                                    data=(monthly_change.to_dict('records')),
                                    style_header={'backgroundColor': 'rgb(30, 30, 30)',
                                                  'fontWeight':'bold',
                                                  'color': colors['text'],
                                                  'textAlign':'center'},
                                    style_cell={
                                        'backgroundColor': 'rgb(50, 50, 50)',
                                        'color': colors['text'],
                                        'textAlign':'center'
                                    },
                                    page_action="native",
                                    page_current= 2,
                                    page_size= 4,
                                  
                                )
                            ],
                            )
                    ],
                    style={'width': '100%', 'display': 'inline-block','marginTop':'10px'},
                    ),
                    
                      ]
           )
        
      ])
    ])
         
            

if __name__ == '__main__':
    app.run_server(debug=False, use_reloader=True,dev_tools_ui=False)






Writing Sarit_app.py


In [0]:
# Below command generate ngrok link... open link that link after program execution done
get_ipython().system_raw('./ngrok http 8050 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

!python Sarit_app.py




http://d459afea.ngrok.io
<IPython.core.display.Javascript object>
<IPython.core.display.Javascript object>
<IPython.core.display.Javascript object>
/usr/local/lib/python3.6/dist-packages/ffn/core.py:27: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)
/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
<IPython.core.display.Javascript object>
Henry Hub Natural Gas Spot Price, Daily (Dollars per Million Btu)
<IPython.core.display.Javascript object>
<class 'pandas.core.frame.DataFrame'>
<IPython.core.display.Javascript object>
<IPython.core.display.Javascript object>


US Coal Prices by Region
Natural Gas Ro